## Setup

In [1]:
import pandas as pd
from epiweeks import Week

In [2]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Domestic Violence Dataset

In [3]:
domestic_violence = pd.read_csv('Policia Nacional/Domestic_Violence_Reports.csv')
domestic_violence

,DEPARTAMENTO,MUNICIPIO,CODIGO DANE,ARMAS MEDIOS,FECHA HECHO,GENERO,GRUPO ETARIO,CANTIDAD
0,ATLÁNTICO,BARRANQUILLA (CT),8001000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
1,BOYACÁ,DUITAMA,15238000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
2,CAQUETÁ,PUERTO RICO,18592000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
3,CASANARE,MANÍ,85139000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
4,CUNDINAMARCA,BOGOTÁ D.C. (CT),11001000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
...,...,...,...,...,...,...,...,...
575716,VALLE,CARTAGO,76147000,SIN EMPLEO DE ARMAS,28/02/2023,FEMENINO,ADULTOS,1
575717,VALLE,GUACARÍ,76318000,SIN EMPLEO DE ARMAS,28/02/2023,FEMENINO,ADULTOS,1
575718,VALLE,GUADALAJARA DE BUGA,76111000,SIN EMPLEO DE ARMAS,28/02/2023,FEMENINO,ADULTOS,1
575719,VALLE,PALMIRA,76520000,SIN EMPLEO DE ARMAS,28/02/2023,FEMENINO,ADULTOS,2


In [4]:
domestic_violence['CODIGO DANE'] = domestic_violence['CODIGO DANE'].apply(lambda x: str(x)[:-3])


In [5]:
# Convert 'FECHA HECHO' to datetime
domestic_violence['FECHA HECHO'] = pd.to_datetime(domestic_violence['FECHA HECHO'], format='%d/%m/%Y')

# Function to calculate epiweek
def calculate_epiweek(date):
    return Week.fromdate(date)

# Apply the function to create a new 'epiweek' column
domestic_violence['epiweek'] = domestic_violence['FECHA HECHO'].apply(calculate_epiweek)

# Pivot the DataFrame to create columns for each epiweek
pivot_df = domestic_violence.pivot_table(index=['CODIGO DANE', 'DEPARTAMENTO', 'MUNICIPIO', 'ARMAS MEDIOS', 'GENERO', 'GRUPO ETARIO'],
                          columns='epiweek',
                          values='CANTIDAD',
                          aggfunc='sum',
                          fill_value=0)

# Reset the index to make 'CODIGO DANE' a column
pivot_df.reset_index(inplace=True)

# Rename the columns for clarity
pivot_df.columns.name = None

# Group by 'MUNICIPIO' and sum all other columns
domestic_violence = pivot_df.groupby('CODIGO DANE').agg({
    'DEPARTAMENTO': lambda x: x.mode().iloc[0],
    #'MUNICIPIO': lambda x: x.mode().iloc[0],
    'ARMAS MEDIOS': lambda x: x.mode().iloc[0],
    'GENERO': lambda x: x.mode().iloc[0],
    'GRUPO ETARIO': lambda x: x.mode().iloc[0],
}).reset_index()

# Sum all other columns (epiweek columns)
for column in pivot_df.columns[6:]:
    domestic_violence[column] = pivot_df.groupby('CODIGO DANE')[column].sum().to_list()

# Replace NaN with 0 in the epiweek columns
domestic_violence.fillna(0, inplace=True)

# Convert CODIGO DANE to numeric and drop rows with string values
domestic_violence['CODIGO DANE'] = pd.to_numeric(domestic_violence['CODIGO DANE'], errors='coerce')
domestic_violence.dropna(inplace=True)

domestic_violence

,CODIGO DANE,DEPARTAMENTO,ARMAS MEDIOS,GENERO,GRUPO ETARIO,200952,201001,201002,201003,201004,...,202252,202301,202302,202303,202304,202305,202306,202307,202308,202309
0,11001.0,CUNDINAMARCA,SIN EMPLEO DE ARMAS,FEMENINO,ADULTOS,40,65,76,73,59,...,299,764,666,795,727,825,744,805,634,288
1,13001.0,BOLÍVAR,ARMA BLANCA / CORTOPUNZANTE,MASCULINO,ADULTOS,4,11,9,10,14,...,19,63,40,49,55,47,72,61,56,21
2,13006.0,BOLÍVAR,CONTUNDENTES,FEMENINO,ADULTOS,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,13030.0,BOLÍVAR,CONTUNDENTES,FEMENINO,ADULTOS,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,13042.0,BOLÍVAR,SIN EMPLEO DE ARMAS,FEMENINO,ADULTOS,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096,97666.0,VAUPÉS,SIN EMPLEO DE ARMAS,FEMENINO,ADULTOS,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1097,99001.0,VICHADA,NO REPORTADO,FEMENINO,ADULTOS,0,0,0,0,1,...,0,0,0,2,1,3,1,0,1,0
1098,99524.0,VICHADA,NO REPORTADO,FEMENINO,ADULTOS,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1099,99624.0,VICHADA,SIN EMPLEO DE ARMAS,FEMENINO,ADULTOS,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Colombian Census Dataset

In [6]:
census = pd.read_csv('Metadata/Census.csv').iloc[:, :53]
census

,Municipality code,Municipality,Population2007,Population2008,Population2009,Population2010,Population2011,Population2012,Population2013,Population2014,...,Householdswithoutwateraccess(%),Householdswithoutinternetaccess(%),Buildingstratification1(%),Buildingstratification2(%),Buildingstratification3(%),Buildingstratification4(%),Buildingstratification5(%),Buildingstratification6(%),NumberofhospitalsperKm2,NumberofhousesperKm2
0,5001,Medellín,2286126,2309689,2331389,2353410,2379920,2410046,2428850,2436071,...,1.44,29.47,12.1117,31.2817,26.0471,9.0279,6.4855,3.7062,1.555263,2102.057895
1,5002,Abejorral,20643,20944,21197,21355,20973,20067,19732,19796,...,21.87,56.27,31.3520,33.8306,2.0439,0.0352,0.0117,0.0000,0.002012,11.945674
2,5004,Abriaquí,2261,2290,2321,2353,2386,2424,2471,2488,...,24.03,62.35,21.5705,39.1675,4.3519,0.0946,0.1892,0.1892,0.000000,2.548276
3,5021,Alejandría,3935,4020,4096,4171,4244,4337,4419,4467,...,17.06,54.61,17.9879,38.5721,4.8215,0.1854,0.0464,0.0464,0.006623,9.092715
4,5030,Amagá,27996,28331,28624,28839,28776,28371,28259,28432,...,3.31,56.12,27.8784,42.0280,3.4086,0.1889,0.1460,0.0172,0.011765,102.752941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,97889,Yavaraté,555,693,841,1003,1164,1275,1349,1334,...,92.93,75.76,6.0606,1.0101,0.0000,0.0000,0.0000,0.0000,0.000000,0.028621
1117,99001,Puerto Carreño,13742,14559,15346,16124,16934,17773,18635,19519,...,32.52,68.68,41.0368,15.3779,2.4037,0.0579,0.0145,0.0145,0.000388,0.432426
1118,99524,La Primavera,5190,5784,6365,6920,7496,8034,8617,9166,...,26.46,72.64,39.3295,5.0284,0.7782,0.0299,0.0000,0.0000,0.000000,0.113272
1119,99624,Santa Rosalía,3167,3378,3586,3846,4067,4325,4535,4635,...,27.58,81.35,50.2627,7.0053,0.2627,0.0000,0.0000,0.0000,0.000254,0.247904


### Merge

In [7]:
# Merge the two DataFrames on municipality code:
df = census.merge(domestic_violence, left_on='Municipality code', right_on='CODIGO DANE')
df

,Municipality code,Municipality,Population2007,Population2008,Population2009,Population2010,Population2011,Population2012,Population2013,Population2014,...,202252,202301,202302,202303,202304,202305,202306,202307,202308,202309
0,5001,Medellín,2286126,2309689,2331389,2353410,2379920,2410046,2428850,2436071,...,77,153,146,166,175,146,174,138,131,54
1,5002,Abejorral,20643,20944,21197,21355,20973,20067,19732,19796,...,0,0,0,2,0,1,0,0,0,0
2,5004,Abriaquí,2261,2290,2321,2353,2386,2424,2471,2488,...,0,1,0,0,0,0,0,0,0,0
3,5021,Alejandría,3935,4020,4096,4171,4244,4337,4419,4467,...,0,0,0,1,0,0,0,0,0,0
4,5030,Amagá,27996,28331,28624,28839,28776,28371,28259,28432,...,3,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096,97666,Taraira,432,540,689,891,1148,1355,1492,1612,...,0,0,0,0,0,0,0,1,0,0
1097,99001,Puerto Carreño,13742,14559,15346,16124,16934,17773,18635,19519,...,0,0,0,2,1,3,1,0,1,0
1098,99524,La Primavera,5190,5784,6365,6920,7496,8034,8617,9166,...,0,0,0,0,0,0,0,0,0,0
1099,99624,Santa Rosalía,3167,3378,3586,3846,4067,4325,4535,4635,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# save the DataFrame as a csv file
df.to_csv('Census_and_Domestic_Violence.csv', index=False)